# Data pipeline form pyspark(databricks) to snowflake

# I - Loading data

 ## 1 - Loading airlines data

In [0]:
#libraries
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql.types import StructType as R, StructField as Fld, DoubleType as Dbl, StringType as Str, IntegerType as Int, DateType as Date, LongType as long
import  pyspark.sql.functions as F
from pyspark.sql import types as t

In [0]:
airlines = spark.read.csv('dbfs:/FileStore/tables/airlines/airlines.csv',schema="IATA_CODE STRING,AIRLINE STRING")
airlines.show(5,False)

+---------+----------------------+
IATA_CODE|AIRLINE |
+---------+----------------------+
IATA_CODE|AIRLINE |
UA |United Air Lines Inc. |
AA |American Airlines Inc.|
US |US Airways Inc. |
F9 |Frontier Airlines Inc.|
+---------+----------------------+
only showing top 5 rows

In [0]:
airlines.printSchema()


root
-- IATA_CODE: string (nullable = true)
-- AIRLINE: string (nullable = true)

In [0]:
airlines.describe().show()

+-------+---------+--------------+
summary|IATA_CODE| AIRLINE|
+-------+---------+--------------+
 count| 15| 15|
 mean| null| null|
 stddev| null| null|
 min| AA| AIRLINE|
 max| WN|Virgin America|
+-------+---------+--------------+

In [0]:
def process_airlines_data(spark,input_data, output_data,snowflake_options,snowflake_table):
  """
  This function reads the airlines file and load into a data frame.
  drop the duplicates if exists and load into snowFlake
  """
  airlines_schema = R([
    Fld("IATA_CODE" ,Str()),
    Fld("AIRLINE",Str()),
    ])
  
  airlines = spark.read.csv(input_data,schema=airlines_schema)
  airlines = airlines.drop_duplicates()
  
  #loading to snowflake
  airlines.write.format("snowflake").options(**snowflake_options).option("dbtable", snowflake_table).mode('append').options(header=True).save()

## 2 - Loading airports data

In [0]:
airports = spark.read.csv('dbfs:/FileStore/tables/airlines/airports.csv', inferSchema=True, header=True)
airports.show(5,False)
airports.printSchema()

+---------+-----------------------------------+-----------+-----+-------+--------+----------+
IATA_CODE|AIRPORT |CITY |STATE|COUNTRY|LATITUDE|LONGITUDE |
+---------+-----------------------------------+-----------+-----+-------+--------+----------+
ABE |Lehigh Valley International Airport|Allentown |PA |USA |40.65236|-75.4404 |
ABI |Abilene Regional Airport |Abilene |TX |USA |32.41132|-99.6819 |
ABQ |Albuquerque International Sunport |Albuquerque|NM |USA |35.04022|-106.60919|
ABR |Aberdeen Regional Airport |Aberdeen |SD |USA |45.44906|-98.42183 |
ABY |Southwest Georgia Regional Airport |Albany |GA |USA |31.53552|-84.19447 |
+---------+-----------------------------------+-----------+-----+-------+--------+----------+
only showing top 5 rows

root
-- IATA_CODE: string (nullable = true)
-- AIRPORT: string (nullable = true)
-- CITY: string (nullable = true)
-- STATE: string (nullable = true)
-- COUNTRY: string (nullable = true)
-- LATITUDE: double (nullable = true)
-- LONGITUDE: double (nullable = true)

In [0]:
airports.describe().show()

+-------+---------+--------------------+--------+-----+-------+------------------+------------------+
summary|IATA_CODE| AIRPORT| CITY|STATE|COUNTRY| LATITUDE| LONGITUDE|
+-------+---------+--------------------+--------+-----+-------+------------------+------------------+
 count| 322| 322| 322| 322| 322| 319| 319|
 mean| null| null| null| null| null|38.981243918495295|-98.37896445141064|
 stddev| null| null| null| null| null| 8.616735581018041|21.523492046498102|
 min| ABE|Aberdeen Regional...|Aberdeen| AK| USA| 13.48345| -176.64603|
 max| YUM|Yuma Internationa...| Yuma| WY| USA| 71.28545| -64.79856|
+-------+---------+--------------------+--------+-----+-------+------------------+------------------+

In [0]:
def process_airports_data(spark,input_data, output_data,snowflake_options,snowflake_table):
  """
  This function reads the airports file and load into a data frame.
  drop the duplicates if exists and load into snowFlake
  """
  airports_schema = R([
    Fld("IATA_CODE" ,Str()),
    Fld("AIRPORT",Str()),
    Fld("CITY" ,Str()),
    Fld("STATE",Str()),
    Fld("COUNTRY" ,Str()),
    Fld("LATITUDE",Dbl()),
    Fld("LONGITUDE",Dbl())
    ])
  
  airports = spark.read.csv(input_data,schema=airports_schema)
  airports = airports.drop_duplicates()
  
  #loading to snowflake
  airports.write.format("snowflake").options(**snowflake_options).option("dbtable", snowflake_table).mode('append').options(header=True).save()

## 3 - Loading flights data

In [0]:
flights = spark.read.csv("dbfs:/FileStore/tables/flights2/", inferSchema=True, header=True)
flights.show(5)


+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
2015| 7| 1| 3| MQ| 3029| N935MQ| DFW| CID| 1148| 1204| 16| 12| 1216| 124| 119| 103| 685| 1359| 4| 1352| 1403| 11| 0| 0| null| null| null| null| null| null|
2015| 7| 1| 3| MQ| 3283| N690MQ| DFW| MLI| 1149| 1221| 32| 12| 1233| 122| 116| 101| 691| 1414| 3| 1351| 1417| 26| 0| 0| null| 0| 0| 26| 0| 0|
2015| 7| 1| 3| DL| 884| N986AT| TPA| LGA| 1149| 1259| 70| 11| 1310| 161| 146| 128| 1010| 1518| 7| 1430| 1525| 55| 0| 0| null| 0| 0| 6| 49| 0|
2015| 7| 1| 3| B6| 265| N593JB| SMF| LGB| 1149| 1151| 2| 15| 1206| 80| 85| 66| 387| 1312| 4| 1309| 1316| 7| 0| 0| null| null| null| null| null| null|
2015| 7| 1| 3| B6| 249| N203JB| DCA| TPA| 1149| 1206| 17| 15| 1221| 137| 129| 112| 814| 1413| 2| 1406| 1415| 9| 0| 0| null| null| null| null| null| null|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
only showing top 5 rows

In [0]:
flights.printSchema()

root
-- YEAR: integer (nullable = true)
-- MONTH: integer (nullable = true)
-- DAY: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- AIRLINE: string (nullable = true)
-- FLIGHT_NUMBER: integer (nullable = true)
-- TAIL_NUMBER: string (nullable = true)
-- ORIGIN_AIRPORT: string (nullable = true)
-- DESTINATION_AIRPORT: string (nullable = true)
-- SCHEDULED_DEPARTURE: integer (nullable = true)
-- DEPARTURE_TIME: integer (nullable = true)
-- DEPARTURE_DELAY: integer (nullable = true)
-- TAXI_OUT: integer (nullable = true)
-- WHEELS_OFF: integer (nullable = true)
-- SCHEDULED_TIME: integer (nullable = true)
-- ELAPSED_TIME: integer (nullable = true)
-- AIR_TIME: integer (nullable = true)
-- DISTANCE: integer (nullable = true)
-- WHEELS_ON: integer (nullable = true)
-- TAXI_IN: integer (nullable = true)
-- SCHEDULED_ARRIVAL: integer (nullable = true)
-- ARRIVAL_TIME: integer (nullable = true)
-- ARRIVAL_DELAY: integer (nullable = true)
-- DIVERTED: integer (nullable = true)
-- CANCELLED: integer (nullable = true)
-- CANCELLATION_REASON: string (nullable = true)
-- AIR_SYSTEM_DELAY: integer (nullable = true)
-- SECURITY_DELAY: integer (nullable = true)
-- AIRLINE_DELAY: integer (nullable = true)
-- LATE_AIRCRAFT_DELAY: integer (nullable = true)
-- WEATHER_DELAY: integer (nullable = true)

In [0]:
def process_flights_data(spark,input_data, snowflake_options,snowflake_table):
  """
  This function reads the flights file and load into a data frame.
  drop the duplicates if exists and load into snowFlake
  """
  flights_schema = R([
    Fld("YEAR" , Int() ),
    Fld("MONTH" , Int() ),
    Fld("DAY" , Int() ),
    Fld("DAY_OF_WEEK" , Int() ),
    Fld("AIRLINE" , Str() ),
    Fld("FLIGHT_NUMBER" , Int() ),
    Fld("TAIL_NUMBER" , Str() ),
    Fld("ORIGIN_AIRPORT" , Str() ),
    Fld("DESTINATION_AIRPORT" , Str() ),
    Fld("SCHEDULED_DEPARTURE" , Int() ),
    Fld("DEPARTURE_TIME" , Int() ),
    Fld("DEPARTURE_DELAY" , Int() ),
    Fld("TAXI_OUT" , Int() ),
    Fld("WHEELS_OFF" , Int() ),
    Fld("SCHEDULED_TIME" , Int() ),
    Fld("ELAPSED_TIME" , Int() ),
    Fld("AIR_TIME" , Int() ),
    Fld("DISTANCE" , Int() ),
    Fld("WHEELS_ON" , Int() ),
    Fld("TAXI_IN" , Int() ),
    Fld("SCHEDULED_ARRIVAL" , Int() ),
    Fld("ARRIVAL_TIME" , Int() ),
    Fld("ARRIVAL_DELAY" , Int() ),
    Fld("DIVERTED" , Int() ),
    Fld("CANCELLED" , Int() ),
    Fld("CANCELLATION_REASON" , Str() ),
    Fld("AIR_SYSTEM_DELAY" , Int() ),
    Fld("SECURITY_DELAY" , Int() ),
    Fld("AIRLINE_DELAY" , Int() ),
    Fld("LATE_AIRCRAFT_DELAY" , Int() ),
    Fld("WEATHER_DELAY" , Int() )
 ])

  
  flights = spark.read.csv(input_data,schema=flights_schema)
  flights = flights.drop_duplicates()
  
  #loading to snowflake
  flights.write.format("snowflake").options(**snowflake_options).option("dbtable", snowflake_table).mode('append').options(header=True).save()

### 4 - Loading Calendar table

In [0]:
def process_calendar(fights,snowflake_options,snowflake_table):
  """
    This function take as input the fight datarame, select dates columns
    add two columns for month and day of the week
    Then load into snowflake
  """
  
  month_lst = ['January', 'Feburary', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
  days = ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday')

  month_name = udf(lambda x: month_lst[int(x%12) - 1], Str())
  day_name = udf(lambda x : days[int(x%7)-1], Str())

  time_table = flights.select("YEAR","MONTH","DAY","DAY_OF_WEEK").drop_duplicates()


  time_table= time_table.withColumn("MONTH_NAME",month_name(col("MONTH")))          
  time_table= time_table.withColumn("DAY_NAME",day_name(col("DAY")))  
  
  #loading to snowflake
  time_table.write.format("snowflake").options(**snowflake_options).option("dbtable", snowflake_table).mode('append').options(header=True).save()


In [0]:
process_calendar(flights,SNOWFLAKE_OPTIONS,'CALENDAR')

## Preparing Data for Modeling

In [0]:
import pandas as pd

data =[]
### Handling duplicates

#airline count
all_airlines = airlines.count()
distinct_arlines = airlines.distinct().count()

#flights count
all_flights = flights.count()
distinct_flights = flights.distinct().count()

#airports count
all_airport = airports.count()
distinct_airport = airports.distinct().count()

print(f'all arlines ={all_airlines}, distinct airlines={distinct_arlines}')
print(f'all flights ={all_flights},  distinct flights={distinct_flights}')
print(f'all airport ={all_airport}, distinct airport={distinct_airport}')

data.append(['airlines',distinct_arlines,0])
data.append(['flights',distinct_flights,0])
data.append(['airport',distinct_airport,0])

data_check=pd.DataFrame(data,columns=['dataset','source_count','destination_count'])
display(data_check)


all arlines =15, distinct airlines=15
all flights =3920766, distinct flights=3920766
all airport =322, distinct airport=322

dataset,source_count,destination_count
airlines,15,0
flights,3920766,0
airport,322,0


In [0]:
print(f' The flight dataset has {len(flights.columns)} columns')

The flight dataset has 31 columns

In [0]:
# spark._jvm.net.snowflake.spark.snowflake.SnowflakeConnectorUtils.enablePushdownSession(spark._jvm.org.apache.spark.sql.SparkSession.builder().getOrCreate())

In [0]:
import os
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

### Defining Snowflake connection options

In [0]:
SNOWFLAKE_OPTIONS = {
    'sfURL': os.environ.get("SNOWFLAKE_URL", "ofa61631.us-east-1.snowflakecomputing.com"),
    'sfAccount': os.environ.get("SNOWFLAKE_ACCOUNT", "ofa61631"),
    'sfUser': os.environ.get("SNOWFLAKE_USER", "TKONCHOU"),
    'sfPassword': os.environ.get("SNOWFLAKE_PASSWORD", "Tida1973"),
    'sfDatabase': os.environ.get("SNOWFLAKE_DATABASE", "INTERVIEW_WH"),
    'sfSchema': os.environ.get("SNOWFLAKE_SCHEMA", "PUBLIC"),
    'sfWarehouse': os.environ.get("SNOWFLAKE_WAREHOUSE", "DATASCIENCE_WH"),
    'sfRole':  os.environ.get("SNOWFLAKE_ROLE", "accountadmin"),
    }

## Loading Data

In [0]:
data_frame = spark.read.format(SNOWFLAKE_SOURCE_NAME).options(**SNOWFLAKE_OPTIONS).option('dbtable', "airports").load()

In [0]:
data_frame

Out[23]: DataFrame[IATA_CODE: string, AIRPORT: string, CITY: string, STATE: string, COUNTRY: string, LATITUDE: double, LONGITUDE: double]

In [0]:
spark.range(5).write.format("snowflake").options(**SNOWFLAKE_OPTIONS).option("dbtable","INTERVIEW_WH").save()
#spark.airports.write.format("snowflake").options(**SNOWFLAKE_OPTIONS).option("dbtable","INTERVIEW_WH").save()

In [0]:
SparkSession.builder.appName(‘Pyspark_snowflake’).getOrCreate()
spark._jvm.net.snowflake.spark.snowflake.SnowflakeConnectorUtils.enablePushdownSession(spark._jvm.org.apache.spark.sql.SparkSession.builder().getOrCreate())

File "<command-2920939853352798>" , line 1 
 SparkSession.builder.appName(‘Pyspark_snowflake’).getOrCreate() 
 ^
 SyntaxError : invalid character in identifier

In [0]:
airports.write.format("snowflake").options(**SNOWFLAKE_OPTIONS).option("dbtable", "AIRPORTS").mode('append').options(header=True).save()

In [0]:
airlines.write.format("snowflake").options(**SNOWFLAKE_OPTIONS).option("dbtable", "AIRLINES").mode('append').options(header=True).save()

In [0]:
flights.write.format("snowflake").options(**SNOWFLAKE_OPTIONS).option("dbtable", "flights").mode('append').options(header=True).save()

In [0]:
airline_count = spark.read.format("snowflake") \
  .options(**SNOWFLAKE_OPTIONS) \
  .option("query",  "SELECT COUNT(*) as cnt FROM public.AIRLINES") \
  .load()

airport_count = spark.read.format("snowflake") \
  .options(**SNOWFLAKE_OPTIONS) \
  .option("query",  "SELECT COUNT(*) as cnt FROM public.AIRPORTS") \
  .load()

flight_count = spark.read.format("snowflake") \
  .options(**SNOWFLAKE_OPTIONS) \
  .option("query",  "SELECT COUNT(*) as cnt FROM public.FLIGHTS") \
  .load()

data_check.loc[data_check.dataset=='airlines','destination_count'] = airline_count.collect()[0][0]
data_check.loc[data_check.dataset=='airport','destination_count'] = airport_count.collect()[0][0]
data_check.loc[data_check.dataset=='flights','destination_count'] = flight_count.collect()[0][0]

display(data_check)

dataset,source_count,destination_count
airlines,15,15
flights,3920766,3920766
airport,322,322
